In [11]:
import pandas as pd

df = pd.read_excel("./files/logs.txt.xlsx",header=None)
df.head(10)


# output_dir = Path("./files").parent / "output"
# output_dir.mkdir(parents=True, exist_ok=True)

# Define Excel file path
# excel_file = Path("./files").parent / "Financial_Sample.xlsx"

# df = pd.read_excel(excel_file)
# column_name = "0"
# unique_values = df[column_name].unique()

# for unique_value in unique_values:
#     df_output = df[df[column_name].str.fullmatch(unique_value)]
#     output_path = output_dir / f"{unique_value}.xlsx"
#     df_output.to_excel(output_path, sheet_name=unique_value[:31], index=False)


    
    
# df.to_excel("./data/healthexp.xlsx", sheet_name="data", index=False)

,0
0,"Timestamp: 2023-10-27 10:00:00, Error: ERR_400"
1,"Timestamp: 2023-10-27 10:00:00, Error: ERR_500"
2,"Timestamp: 2023-10-27 10:00:00, Error: INFO_200"
3,"Timestamp: 2023-10-27 10:00:00, Error: INFO_200"
4,"Timestamp: 2023-10-27 10:00:00, Error: WARN_101"
5,"Timestamp: 2023-10-27 10:00:00, Error: INFO_200"
6,"Timestamp: 2023-10-27 10:00:00, Error: ERR_500"
7,"Timestamp: 2023-10-27 10:00:00, Error: INFO_200"
8,"Timestamp: 2023-10-27 10:00:00, Error: INFO_200"
9,"Timestamp: 2023-10-27 10:00:00, Error: WARN_101"


In [ ]:

range_value = 100000
for i in range(int(df.shape[0]/range_value)):
    df_to_excel = df[i*range_value:(i+1)*range_value]
    print("i",i)
#     print(df_to_excel)
    df_to_excel.to_excel(f"./files/log{i}.xlsx", sheet_name="data", index=False)


i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
